In [15]:
### PYTHON IMPORTS
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from random import shuffle
import librosa



In [101]:
""" Convolutional Neural Network.

Build and train a convolutional neural network with TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import division, print_function, absolute_import
import os
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Parameters
learning_rate = 0.001
num_steps = 200
batch_size = 128
display_step = 10
# parameters

num_classes = 10 #numberOfClasses

num_input = 128*44*1

# saver train parameters
useCkpt = False
checkpoint_step = 5
checkpoint_dir = os.getcwd()+'\\checkpoint\\'


# Network Parameters
num_input = 28*28 # MNIST data input (img shape: 28*28)
#num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print("conv2")
    print(conv2.shape) # (?, 7, 7, 64)
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    print("conv3")
    print(conv3.shape) # (?, 7, 7, 64)
    
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    print("conv4")
    print(conv4.shape) # (?, 7, 7, 64)
    
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = maxpool2d(conv5, k=2)
    print("conv5")
    print(conv5.shape) # (?, 7, 7, 64)
    
    return conv5
    

def mlp(convFinal, weights, biases, dropout):
    #(?, 3136)
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(convFinal, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("fc1")
    print(fc1.shape)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

def elm(convFinal, weights, biases, dropout):
    #conv5Shape = convFinal.get_shape().as_list()
    conv5Shape = convFinal.get_shape().as_list()
    print("CONC")
    print(conv5Shape)

    print("ELM")
    print(convFinal.shape) # 10
    fc1 = tf.reshape(convFinal, [-1, weights['elm_wd1'].get_shape().as_list()[0]])
    print(fc1.shape) # 512

    fc1 = tf.add(tf.matmul(fc1,weights['elm_wd1']), biases['elm_bd1'])
    print(fc1.shape) # 256

    fc1 = tf.nn.relu(fc1)
    print(fc1.shape) # 256

    return fc1

def elm_prediction(convFinal, weights, biases, dropout):
    #conv5Shape = convFinal.get_shape().as_list()
    
    fc1 = tf.reshape(convFinal, [-1, weights['elm_wd1'].get_shape().as_list()[0]]) #conv5Shape[1] * conv5Shape[2] * conv5Shape[3]])
    fc1 = tf.add(tf.matmul(fc1,weights['elm_wd1']), biases['elm_bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    out = tf.add(tf.matmul(fc1,weights['elm_out']), biases['elm_out'])

    prediction = tf.nn.softmax(out)
    
    return prediction

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    
    'wc3': tf.Variable(tf.random_normal([5, 5, 64, 128])),
    
    'wc4': tf.Variable(tf.random_normal([5, 5, 128, 256])),
    
    'wc5': tf.Variable(tf.random_normal([5, 5, 256, 512])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([512, 256])),
    'elm_wd1': tf.Variable(tf.random_normal([512, 256])),

    # 1024 inputs, 10 outputs (class prediction)
    'elm_out': tf.Variable(tf.random_normal([256, num_classes])),
    'out': tf.Variable(tf.random_normal([256, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([256])),
    'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([256])),
    'elm_bd1': tf.Variable(tf.random_normal([256])),

    'out': tf.Variable(tf.random_normal([num_classes])),
    'elm_out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
mlp_conv = conv_net(X, weights, biases, keep_prob)
logits = mlp(mlp_conv, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#ELM
elm_convFinal = conv_net(X,weights,biases,keep_prob)

elm_fc1 = elm(elm_convFinal, weights, biases, dropout)
output_weights = tf.matrix_solve_ls(elm_fc1,Y, l2_regularizer=0.0,fast=False,name=None)
weights_output = tf.assign( weights['elm_out'], output_weights, validate_shape=None, use_locking=None, name=None)
elm_logits = elm_prediction(elm_convFinal,weights,biases, dropout)

# evaluate model
correct_pred_elm = tf.equal(tf.argmax(elm_logits,1),tf.argmax(Y,1))
accuracy_elm = tf.reduce_mean(tf.cast(correct_pred_elm,tf.float32))

# Initialize the variables (i.e. assign their default value)
hidden_num = 400
input_len = 512
output_len = 10

_W = tf.Variable(tf.random_normal([input_len, hidden_num]),trainable=False, dtype=tf.float32)
_b = tf.Variable(tf.random_normal([hidden_num]),trainable=False, dtype=tf.float32)
_beta = tf.Variable( tf.zeros([hidden_num, output_len]),trainable=False, dtype=tf.float32)
init = tf.global_variables_initializer()



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
conv2
(?, 7, 7, 64)
conv3
(?, 4, 4, 128)
conv4
(?, 2, 2, 256)
conv5
(?, 1, 1, 512)
fc1
(?, 512)
conv2
(?, 7, 7, 64)
conv3
(?, 4, 4, 128)
conv4
(?, 2, 2, 256)
conv5
(?, 1, 1, 512)
CONC
[None, 1, 1, 512]
ELM
(?, 1, 1, 512)
(?, 512)
(?, 256)
(?, 256)


In [102]:


class ELM(object):
  def __init__(self, sess, batch_size, input_len, hidden_num, output_len, w, b, beta):

    self._sess = sess 
    self._batch_size = batch_size
    self._input_len = input_len
    self._hidden_num = hidden_num
    self._output_len = output_len 

    # for train
    self._x0 = tf.placeholder(tf.float32, [self._batch_size, self._input_len])
    self._t0 = tf.placeholder(tf.float32, [self._batch_size, self._output_len])

    # for test
    self._x1 = tf.placeholder(tf.float32, [None, self._input_len])
    self._t1 = tf.placeholder(tf.float32, [None, self._output_len])

    self._W = w
    self._b = b
    self._beta = beta
    #self._var_list = [self._W, self._b, self._beta]

    self.H0 = tf.matmul(self._x0, self._W) + self._b # N x L
    self.H0_T = tf.transpose(self.H0)

    self.H1 = tf.matmul(self._x1, self._W) + self._b # N x L
    self.H1_T = tf.transpose(self.H1)

    # beta analytic solution : self._beta_s (K x O)
    if self._input_len < self._hidden_num: # L < K
      identity = tf.constant(np.identity(self._hidden_num), dtype=tf.float32)
      self._beta_s = tf.matmul(tf.matmul(tf.matrix_inverse(
        tf.matmul(self.H0_T, self.H0) + identity), 
        self.H0_T), self._t0)
      # _beta_s = (H_T*H + I/om)^(-1)*H_T*T
    else:
      identity = tf.constant(np.identity(self._batch_size), dtype=tf.float32)
      self._beta_s = tf.matmul(tf.matmul(self.H0_T, tf.matrix_inverse(
        tf.matmul(self.H0, self.H0_T)+identity)), self._t0)
      # _beta_s = H_T*(H*H_T + I/om)^(-1)*T
    
    self._assign_beta = self._beta.assign(self._beta_s)
    self._fx0 = tf.matmul(self.H0, self._beta)
    self._fx1 = tf.matmul(self.H1, self._beta)

    self._cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self._fx0, labels=self._t0))

    # for the mnist test
    self._correct_prediction = tf.equal(tf.argmax(self._fx1,1), tf.argmax(self._t1,1))
    self._accuracy = tf.reduce_mean(tf.cast(self._correct_prediction, tf.float32))

  def feed(self, x, t):
      
    self._sess.run(self._assign_beta, {self._x0:x, self._t0:t})

  def test(self, x, t=None):
    if t is not None :
      print("Accuracy: {:.9f}".format(self._sess.run(self._accuracy, {self._x1:x, self._t1:t})))
    else :
      return self._sess.run(self._fx1, {self._x1:x})

In [104]:
# Basic tf setting
tf.set_random_seed(2016)

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    """for step in range(100): #, 20+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

            print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                          Y: mnist.test.labels[:256],
                                          keep_prob: 1.0}))
    """
    print("Optimization Finished! - Do ELM")
    elm_batchSize_train = 5000
    
    tesee1 = sess.run(tf.reshape(elm_convFinal, [-1, 512]), feed_dict={X: mnist.test.images, keep_prob:1})
    
    elm = ELM(sess, elm_batchSize_train, input_len, hidden_num, output_len,_W,_b,_beta)

    # one-step feed-forward training
    print("Accuracy on test before")
    elm.test(tesee1, mnist.test.labels)  
    
    # TRAIN
    tesee2 = sess.run(tf.reshape(elm_convFinal, [-1, 512]), feed_dict={X: mnist.train.images[:elm_batchSize_train], Y: mnist.train.labels[:elm_batchSize_train], keep_prob: 1})
    #_, train_y = mnist.train.next_batch(batch_size)
    elm.feed(tesee2, mnist.train.labels[:elm_batchSize_train])
    #print("tes1 after")
    #print(tesee1)
    # testing
    # Accuracy on test after
    print("Accuracy on training after")
#    elm.test(mnist.test.images, mnist.test.labels)
    elm.test(tesee2, mnist.train.labels[:elm_batchSize_train])  
    print("Accuracy on test after")
    tesee3 = sess.run(tf.reshape(elm_convFinal, [-1, 512]), feed_dict={X: mnist.test.images, keep_prob:1})

    elm.test(tesee3, mnist.test.labels)  
    
    print("ELM DONE")
    
    # Calculate accuracy for 256 MNIST test images
        

Step 0, Minibatch Loss= 994549888.0000, Training Accuracy= 0.086
Testing Accuracy: 0.109375
Step 1, Minibatch Loss= 470275264.0000, Training Accuracy= 0.109
Testing Accuracy: 0.105469
Step 10, Minibatch Loss= 198766144.0000, Training Accuracy= 0.273
Testing Accuracy: 0.320313
Step 20, Minibatch Loss= 89909432.0000, Training Accuracy= 0.414
Testing Accuracy: 0.414063
Step 30, Minibatch Loss= 31522372.0000, Training Accuracy= 0.602
Testing Accuracy: 0.617188
Step 40, Minibatch Loss= 31616354.0000, Training Accuracy= 0.664
Testing Accuracy: 0.667969
Step 50, Minibatch Loss= 15760748.0000, Training Accuracy= 0.695
Testing Accuracy: 0.738281
Step 60, Minibatch Loss= 17821348.0000, Training Accuracy= 0.781
Testing Accuracy: 0.792969
Step 70, Minibatch Loss= 12696998.0000, Training Accuracy= 0.812
Testing Accuracy: 0.8125
Step 80, Minibatch Loss= 12453655.0000, Training Accuracy= 0.797
Testing Accuracy: 0.808594
Step 90, Minibatch Loss= 8742920.0000, Training Accuracy= 0.789
Testing Accuracy:

In [105]:
import numpy
import PIL

In [121]:
### LOAD ALL FILE PATHS TO ARRAY

cwd = os.getcwd()
dataPath = 'data\FRUIT_360\Training'
pathToData = os.path.join(cwd, dataPath)


fruitFiles = []
labelNames = []
classIndex = -1

# Find all folders inside the data folder
for classFile in os.listdir(pathToData):
    # For all folders found that match the wordsToTrain
    print(classFile)
    classIndex += 1
    labelNames.append(classFile)
    # Check inside every folder
    pathToClass =  os.path.join(pathToData, classFile)
        # Loop through all files found
    for audioFile in os.listdir(pathToClass):
        # If the file format is an audio file
            # append the file path and the correct label to the audio files list
        fruitFiles.append([os.path.join(pathToClass,audioFile),classIndex])
            
shuffle(fruitFiles)
numberOfClasses = classIndex+1
             
print("Number of classes: " + str(numberOfClasses))      
print(len(fruitFiles))

Apple Red 1
Avocado
Banana
Kiwi
Lemon
Limes
Orange
Pear
Pineapple
Strawberry
Number of classes: 10
4564


In [122]:
# LOAD ALL DATA TO ARRAYS

import matplotlib.pyplot as plt
import librosa.display

## Cross validation array structure

# Note - nearly 1700 datapoints for each class. 
# If limit data is used, reduce the number of files by 80%
limitData = False
percentOfData = 0.1

dataPoints = len(fruitFiles)
if(limitData):
    dataPoints = int((dataPoints*percentOfData))

# Set size of training
trainingSize = int(0.65 * dataPoints)
# Set size of validation
validationSize = int(0.25 * dataPoints)
# Rest should then be the testing set
testingSize = int(dataPoints - (trainingSize + validationSize))

print("Number of datapoints are: ",dataPoints)
print("Training samples: ",trainingSize)
print("Validation samples: ",validationSize)
print("Testing samples: ", testingSize)
print("Sum of all sets is ",trainingSize+validationSize+testingSize)

# Streams - link holder
#https://blog.francoismaillet.com/epic-celebration/
#https://stackoverflow.com/questions/42625286/how-to-process-audio-stream-in-realtime

Number of datapoints are:  4564
Training samples:  2966
Validation samples:  1141
Testing samples:  457
Sum of all sets is  4564


In [132]:
# CHANGE DATA TO ARRAY
def fileObjectToDataObjectFRUIT(fileObject):
    img = PIL.Image.open(fileObject[0]).convert("RGB")
    img = img.resize((32, 32), PIL.Image.ANTIALIAS)
    imgarr = np.divide(numpy.array(img),255)
    
    
    targetVector = np.zeros((1,numberOfClasses))[0]
    targetVector[fileObject[1]] = 1
    
    dataObject = {
        'filename' : fileObject[0],
        'data' : imgarr,
        'label' : targetVector,
        'labelName' : labelNames[fileObject[1]]
    }
    
    return dataObject

trainingArray = []
validationArray = []
testingArray = []

# Load training data   # trainingSize
for i in range(trainingSize):
    newObject = fileObjectToDataObjectFRUIT(fruitFiles[i])
    trainingArray.append(newObject)
    
for i in range(validationSize):
    # Get the correct index
    startIndex = i + trainingSize
    newObject = fileObjectToDataObjectFRUIT(fruitFiles[startIndex])

    validationArray.append(newObject)

for i in range(testingSize):
    # Get the correct index
    startIndex = i + trainingSize + validationSize
    newObject = fileObjectToDataObjectFRUIT(fruitFiles[startIndex])
    testingArray.append(newObject)


In [133]:
print(len(trainingArray))

2966


In [134]:

def getArrInfo(anArray, arrName):
    print("Info about dataset " +arrName)
    infoHolder = np.zeros((1,numberOfClasses))[0]
    for i in range(len(anArray)):
        infoHolder += anArray[i]['label']
    
    infoHolder = infoHolder/len(anArray)
    for i in range(numberOfClasses):
        print(str("{:.2%}".format(infoHolder[i])) + " of the dataset is of class " + labelNames[i] )
    print("")
    
getArrInfo(trainingArray, "Training array")    
getArrInfo(validationArray, "Validation array")        
getArrInfo(testingArray, "Testing array")        

Info about dataset Training array
10.62% of the dataset is of class Apple Red 1
9.31% of the dataset is of class Avocado
10.32% of the dataset is of class Banana
10.32% of the dataset is of class Kiwi
5.53% of the dataset is of class Lemon
10.69% of the dataset is of class Limes
10.79% of the dataset is of class Orange
10.79% of the dataset is of class Pear
10.72% of the dataset is of class Pineapple
10.92% of the dataset is of class Strawberry

Info about dataset Validation array
10.96% of the dataset is of class Apple Red 1
9.47% of the dataset is of class Avocado
10.34% of the dataset is of class Banana
10.43% of the dataset is of class Kiwi
5.43% of the dataset is of class Lemon
11.04% of the dataset is of class Limes
10.43% of the dataset is of class Orange
11.22% of the dataset is of class Pear
10.08% of the dataset is of class Pineapple
10.60% of the dataset is of class Strawberry

Info about dataset Testing array
11.38% of the dataset is of class Apple Red 1
9.41% of the datase

In [135]:
print(trainingArray[0]['data'].shape)

(32, 32, 3)
